In [4]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner
from keras_tuner.tuners import RandomSearch

In [5]:
# Import the air_quality_index data.
df = pd.read_csv('Real_Combine-Copy1.csv')

In [6]:
# Check the data
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [14]:
df.isnull().sum()

T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    1
dtype: int64

In [15]:
df = df.dropna()

In [16]:
df.isnull().sum()

T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    0
dtype: int64

In [17]:
X = df.iloc[:,:-1] # independent features.
y = df.iloc[:,-1] # dependent feature.

In [20]:
def build_model(hp): # here hp in the parameter we will be providing.
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)): # Hidden layer range will be 2:20.\
        model.add(layers.Dense(units = hp.Int('units_'+str(i),
                                             min_value = 32,
                                             max_value = 512,
                                             step = 32),
                              activation = 'relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2,1e-3,1e-4])),
                 loss = 'mean_absolute_error',
                 metrics = ['mean_absolute_error'])
    return model

In [21]:
tuner = RandomSearch(
                    build_model,
                    objective = 'val_mean_absolute_error',
                    max_trials = 5,
                    executions_per_trial = 3,
                    directory = 'project1',
                    project_name = 'Ai Quality Index')

In [22]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=5)

In [24]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 06s]
val_mean_absolute_error: 58.66040802001953

Best val_mean_absolute_error So Far: 58.18957010904948
Total elapsed time: 00h 00m 37s
INFO:tensorflow:Oracle triggered exit


In [25]:
tuner.results_summary()

Results summary
Results in project1\Ai Quality Index
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 8
units_0: 480
units_1: 192
learning_rate: 0.001
units_2: 32
units_3: 64
units_4: 192
units_5: 224
units_6: 64
units_7: 128
units_8: 128
units_9: 96
units_10: 448
units_11: 96
units_12: 288
units_13: 320
units_14: 416
units_15: 128
units_16: 320
units_17: 224
Score: 58.18957010904948
Trial summary
Hyperparameters:
num_layers: 8
units_0: 192
units_1: 64
learning_rate: 0.01
units_2: 416
units_3: 480
units_4: 384
units_5: 192
units_6: 480
units_7: 64
units_8: 512
units_9: 64
units_10: 320
units_11: 448
units_12: 384
units_13: 64
units_14: 128
units_15: 320
units_16: 352
units_17: 256
Score: 58.66040802001953
Trial summary
Hyperparameters:
num_layers: 17
units_0: 128
units_1: 224
learning_rate: 0.01
units_2: 160
units_3: 224
units_4: 32
units_5: 320
units_6: 128
units_7: 64
units_8: 416
units_9: 320
units_10: 320
units_11: 192
units_12: 192
units_13: 64
units_14: 384
units